## LM

#### Training

In [ ]:
import tensorflow as tf
import time
import sys
sys.path.append("../src")
from tf_levenberg_marquardt import levenberg_marquardt as lm

In [ ]:
normalizer = tf.keras.layers.Normalization()
normalizer.adapt(X_train)
X_train_norm = normalizer(X_train)

In [ ]:
input_size = 8184
batch_size = 1000

x_train_norm = tf.cast(X_train_norm, tf.float32)
y_train = tf.expand_dims(tf.cast(Y_train, tf.float32), axis=-1)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))
train_dataset = train_dataset.shuffle(input_size)
train_dataset = train_dataset.batch(batch_size).cache()
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(1)
])

model.summary()

model_wrapper = lm.ModelWrapper(
    tf.keras.models.clone_model(model))

model_wrapper.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=10),
    loss=lm.MeanSquaredError())

In [ ]:
print("Train using Levenberg-Marquardt")
t2_start = time.perf_counter()
model_wrapper.fit(generator, epochs=100)
t2_stop = time.perf_counter()
print("Elapsed time: ", t2_stop - t2_start)

In [ ]:
len(y_train.numpy().reshape(1, -1)[0]), len(model_wrapper.predict(x_train_norm).reshape(1, -1)[0])

In [ ]:
print("Plot results")
plot_size = (30, 20)
plt.scatter(model_wrapper.predict(x_train_norm).reshape(1, -1)[0], y_train.numpy().reshape(1, -1)[0])
plt.show()

In [ ]:
model_wrapper.predict(x_train_norm)

#### Validation

In [ ]:
x_test_norm = tf.cast(normalizer(X_test), tf.float32)
y_test = tf.expand_dims(tf.cast(Y_test, tf.float32), axis=-1)
y_test_predict = model_wrapper.predict(x_test_norm)

In [ ]:
print("Plot results")
plot_size = (30, 20)
plt.scatter(y_test_predict.reshape(1, -1)[0], y_test.numpy().reshape(1, -1)[0])
plt.show()

In [ ]:
df_validation = df_train_dropna[index_of_x_days_ago_imputed:][features]
df_validation['predicted_KW'] = y_test_predict
df_validation['KW'] = y_test.numpy()
df_validation['residuals'] = df_validation['KW'] - df_validation['predicted_KW']

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df_validation['KW'], label="Test")
plt.plot(df_validation['predicted_KW'], label="Prediction")
plt.plot(df_validation['residuals'], label="residuals")
plt.legend(loc = 'best')

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df_validation['KW'], label="Test")
plt.plot(df_validation['predicted_KW'], label="Prediction")
plt.plot(df_validation['residuals'], label="residuals")
plt.legend(loc = 'best')

#### Experiments

[Keras Time Series Forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape, date_x_days_ago, df_train[:index_of_x_days_ago].shape

In [ ]:
X_train_rnn = df_train_dropna[:index_of_x_days_ago_imputed]['month_sin', 'month_cos', 'TEMP_AIR'].to_numpy()
Y_train_rnn = df_train_dropna[:index_of_x_days_ago_imputed]['KW'].values
X_test_rnn = df_train_dropna[index_of_x_days_ago_imputed:][features].to_numpy()
Y_test_rnn = df_train_dropna[index_of_x_days_ago_imputed:]['KW'].values

In [ ]:
X_train = df_train_dropna[:index_of_x_days_ago_imputed][features].to_numpy()
Y_train = df_train_dropna[:index_of_x_days_ago_imputed]['KW'].values
X_test = df_train_dropna[index_of_x_days_ago_imputed:][features].to_numpy()
Y_test = df_train_dropna[index_of_x_days_ago_imputed:]['KW'].values

In [ ]:
index_of_x_days_ago_imputed = df_train_dropna.index.get_loc(date_x_days_ago, method='nearest') # index of x days ago from the last date we have in the dataset
date_x_days_ago, index_of_x_days_ago_imputed

In [ ]:
features = ['KW_x_days_ago_same_hour', 'KW_x_days_ago_1_hour_before', 'TEMP_AIR_x_days_ago', 'month_sin', 'month_cos']

In [ ]:
date_x_days_ago = max(df_freq.index.date) - dt.timedelta(days = predict_days) # x is the number of days in future that we want to predict
index_of_x_days_ago = df_freq.index.get_loc(date_x_days_ago, method='nearest') # index of x days ago from the last date we have in the dataset

date_x_days_ago, index_of_x_days_ago

In [ ]:
# 5 (predict days forward) days ago
df_train.loc[:, ('KW_x_days_ago_same_hour')] = df_train['KW'].shift(periods=predict_days*data_frequency_num_points)
df_train.loc[:, ('KW_x_days_ago_1_hour_before')] = df_train['KW'].shift(periods=predict_days*data_frequency_num_points-1)
df_train.loc[:, ('TEMP_AIR_x_days_ago')] = df_train['TEMP_AIR'].shift(periods=predict_days*data_frequency_num_points)
# display data for x days from the first datapoint we have
# thats when we have the first row with all past data points 
df_train[predict_days*data_frequency_num_points: predict_days*data_frequency_num_points+5]

In [ ]:
# encode cyclic features (month and year) 
def cyclic_transformation(column):
    import math
    max_value = column.max()
    sin_values = [math.sin((2*math.pi*x)/max_value) for x in list(column)]
    cos_values = [math.cos((2*math.pi*x)/max_value) for x in list(column)]
    return sin_values, cos_values

In [ ]:
df_train['month'] = df_train.index.month
df_train['month_x_days_ago'] = df_train['month'].shift(periods=predict_days*data_frequency_num_points)
df_train.tail()

In [ ]:
# encode cyclic features (week, month, hour?)
sin_values, cos_values = cyclic_transformation(df_train['month_x_days_ago'])
df_train['month_x_days_ago_sin'] = sin_values
df_train['month_x_days_ago_cos'] = cos_values

In [ ]:
# encode cyclic features (week, month, hour?)
sin_values, cos_values = cyclic_transformation(df_train.index.month)
df_train['month_sin'] = sin_values
df_train['month_cos'] = cos_values

In [ ]:
# Test cell to experiment
# from the plot it appears that January is closer to February than February is to March: it's the same distance waaaaaa

def cyclic_transformation_test(column):
    import math
    max_value = max(column)
    sin_values = [math.sin((2*math.pi*x)/max_value) for x in list(column)]
    cos_values = [math.cos((2*math.pi*x)/max_value) for x in list(column)]
    return sin_values, cos_values
month_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
sin, cos = cyclic_transformation_test(month_values)
test = pd.DataFrame({'month': month_values, 'sin': sin, 'cos': cos})

test['distance'] = np.sqrt(np.power(test['sin'].shift()-test['sin'],2)+ np.power(test['cos'].shift()-test['cos'],2))

display(test)

plot_size = (30, 10)
fig, ax = plt.subplots(figsize=plot_size)
plt.title("Month sin and cos")
sns.scatterplot(x='sin', y='cos', data=test)

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [ ]:
df = pd.read_csv(csv_path)
# Slice [start:stop:step], starting from index 5 take every 6th record.
df = df[5::6]

date_time = pd.to_datetime(df.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

In [ ]:
plot_cols = ['T (degC)', 'p (mbar)', 'rho (g/m**3)']
plot_features = df[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

plot_features = df[plot_cols][:480]
plot_features.index = date_time[:480]
_ = plot_features.plot(subplots=True)